In [21]:
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from unidecode import unidecode
import unidecode

In [3]:
#Cargmos CSV Ciudades

In [5]:
v

In [34]:
datos_csv.shape

(86, 7)

In [12]:
#Quitamos acentos de todos lados
def quitar_acentos(valor):
    """Función para quitar acentos de una cadena de texto."""
    if isinstance(valor, str):  # Solo aplicar a cadenas de texto
        return unidecode.unidecode(valor)  # Corrección aquí
    return valor

# Aplicamos la función quitar_acentos a todas las columnas del DataFrame
for columna in datos_csv.columns:
    datos_csv[columna] = datos_csv[columna].apply(quitar_acentos)


/var/folders/68/6zkv8z4j4rl6ds90br2rb7ph0000gn/T/ipykernel_771/1075267683.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_csv[columna] = datos_csv[columna].apply(quitar_acentos)


#### Seleccionamos Ciudades

Quitaremos = ['Puerto Madero, Chiapas','Nuevo Vallarta, Nayarit','Puerto Madero, Chiapas''Punta Chivato, Baja California Sur','Loreto, Baja California Sur','La Paz, Baja California Sur','Cabo San Lucas, Baja California Sur
','Isla San Marcos, Baja California Sur','Isla de Cedros, Baja California','El Sauzal, Baja California','Sasabe, Sonora','Naco, Sonora','Ojinaga, Chihuahua','Nuevo Progreso, Tamaulipas','Reynosa, Tamaulipas','Ciudad Camargo, Tamaulipas','Diaz Ordaz, Tamaulipas','Ciudad Miguel Aleman, Tamaulipas','El Mezquital, Durango','Zamora, Michoacan','Papantla, Veracruz','Tultitlan, Estado de Mexico','Apizaco, Tlaxcala','Las Choapas, Veracruz']

'San Gregorio Chamicico, Oaxaca','Chahuitea, Oaxaca'

Playa del Carmen, Quintana Roo
Cancun, Quintana Roo
Cozumel, Quintana Roo
Francisco Villa, Quintana Roo
Benito Juarez, Chihuahua
Pichilingue, Baja California Sur


In [32]:
#Quitamos esas ciudades que no van con Casillas,2006
# Lista de ciudades a eliminar
ciudades_eliminar = ['Puerto Madero, Chiapas','Nuevo Vallarta, Nayarit','Puerto Madero, Chiapas',
                     'Punta Chivato, Baja California Sur','Loreto, Baja California Sur',
                     'La Paz, Baja California Sur','Cabo San Lucas, Baja California Sur',
                     'Isla San Marcos, Baja California Sur','Isla de Cedros, Baja California',
                     'El Sauzal, Baja California','Sasabe, Sonora','Naco, Sonora',
                     'Ojinaga, Chihuahua','Nuevo Progreso, Tamaulipas','Reynosa, Tamaulipas',
                     'Ciudad Camargo, Tamaulipas','Diaz Ordaz, Tamaulipas',
                     'Ciudad Miguel Aleman, Tamaulipas','El Mezquital, Durango','Zamora, Michoacan',
                     'Papantla, Veracruz','Tultitlan, Estado de Mexico','Apizaco, Tlaxcala',
                     'Las Choapas, Veracruz','San Gregorio Chamicico, Oaxaca','Chahuitea, Oaxaca']

# Extendemos la lista original de ciudades a eliminar con las nuevas ciudades
ciudades_eliminar.extend([
    'Playa del Carmen, Quintana Roo', 'Cancun, Quintana Roo', 'Cozumel, Quintana Roo',
    'Francisco Villa, Quintana Roo', 'Benito Juarez, Chihuahua', 'Pichilingue, Baja California Sur',
    'San Gregorio Chamicico, Oaxaca', 'Chahuitea, Oaxaca'
])

# Filtramos el DataFrame para excluir las filas cuyos valores en la columna 'ciudad' estén en ciudades_eliminar
datos_csv = datos_csv[~datos_csv['ciudad'].isin(ciudades_eliminar)]

In [14]:
#Quitamos Repetidos
# Eliminar duplicados basados en la columna 'Referencia Geográfica'
datos_csv = datos_csv.drop_duplicates(subset='ciudad', keep='first')


In [16]:
# Crear una nueva columna 'Estado' extrayendo el texto después de la coma
datos_csv['estado'] = datos_csv['ciudad'].apply(lambda x: x.split(', ')[1] if ', ' in x else '')


/var/folders/68/6zkv8z4j4rl6ds90br2rb7ph0000gn/T/ipykernel_771/2267768364.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_csv['estado'] = datos_csv['ciudad'].apply(lambda x: x.split(', ')[1] if ', ' in x else '')


In [17]:
# Definir las listas de estados para cada ruta, asegurándose de que todas las entradas están correctamente formateadas como strings
estados_ruta_golfo = ['Tabasco', 'Nuevo Leon', 'Veracruz', 'Tamaulipas']
estados_ruta_pacifico = ['Guerrero', 'Michoacan', 'Jalisco', 'Nayarit', 'Sinaloa', 'Sonora', 'Baja California', 'Baja California Sur','Colima']
estados_ruta_centro = ['San Luis Potosi','Estado de Mexico', 'Tlaxcala', 'Ciudad de Mexico', 'Puebla', 'Morelos', 'Hidalgo', 'Zacatecas', 'Durango', 'Chihuahua', 'Coahuila', 'Queretaro', 'Aguascalientes', 'Guanajuato']
ruta_sur = ['Chiapas', 'Oaxaca', 'Tabasco', 'Campeche', 'Quintana Roo', 'Yucatan']

# Actualizar la función para asignar la ruta basada en el estado
def asignar_ruta(estado):
    estado = estado.strip()  # Eliminar espacios en blanco al principio y al final
    if estado in estados_ruta_golfo:
        return 'Ruta del Golfo'
    elif estado in estados_ruta_pacifico:
        return 'Ruta del Pacifico'
    elif estado in estados_ruta_centro:
        return 'Ruta Centro'
    elif estado in ruta_sur:
        return 'Ruta Sur'
    else:
        return 'Ruta no identificada'  # Para estados que no coinciden con las listas

# Aplicar la función al DataFrame para crear una nueva columna 'Ruta'
datos_csv['ruta'] = datos_csv['estado'].apply(asignar_ruta)

/var/folders/68/6zkv8z4j4rl6ds90br2rb7ph0000gn/T/ipykernel_771/353168208.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_csv['ruta'] = datos_csv['estado'].apply(asignar_ruta)


In [18]:
# Reordenar las columnas para colocar 'Estado' después de 'Ciudad'
column_order = ['ciudad', 'estado', 'ruta', 'latitud', 'longitud']

# Aplicar el nuevo orden de columnas al DataFrame
datos_csv = datos_csv[column_order]

In [20]:
# Obtenemos los valores únicos de la columna 'ruta'
valores_unicos_ruta = datos_csv['ruta'].unique()

# Imprimimos los valores únicos
print(valores_unicos_ruta)


['Ruta Centro' 'Ruta del Pacifico' 'Ruta del Golfo' 'Ruta Sur']


### Geolocate Addresses

In [24]:
# Initialize the geolocator using OSM Nominatim
geolocator = Nominatim(user_agent="measurement", timeout=10)

# Define a function to get latitude and longitude
def get_lat_lon(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return '', ''  # Return empty strings if location is not found
    except AttributeError:
        return '', ''

# Apply the function to the 'City' column and split the results into 'Latitude' and 'Longitude' columns
datos_csv[['latitud', 'longitud']] = datos_csv['ciudad'].apply(lambda x: pd.Series(get_lat_lon(x)))

In [33]:
datos_csv

,ciudad,estado,ruta,latitud,longitud,codigo_ruta,etiqueta
93,"Irapuato, Guanajuato",Guanajuato,Ruta Centro,20.675876,-101.352105,C,C1
15,"Durango, Durango",Durango,Ruta Centro,24.022008,-104.654741,C,C2
91,"Salamanca, Guanajuato",Guanajuato,Ruta Centro,20.652325,-101.166842,C,C3
48,"Leon, Guanajuato",Guanajuato,Ruta Centro,21.079256,-101.58977,C,C4
47,"Aguascalientes, Aguascalientes",Aguascalientes,Ruta Centro,21.880487,-102.296719,C,C5
...,...,...,...,...,...,...,...
95,"Comitan, Chiapas",Chiapas,Ruta Sur,16.24411,-92.134851,S,S13
94,"Palenque, Chiapas",Chiapas,Ruta Sur,17.453161,-91.812124,S,S14
63,"Campeche, Campeche",Campeche,Ruta Sur,19.838121,-90.519825,S,S15
66,"Chetumal, Quintana Roo",Quintana Roo,Ruta Sur,18.508174,-88.297144,S,S17


  ### Detectamos Coordenadas Incorrectas y las metemos manualmente


In [35]:
# Ajustar latitud y longitud para 'Juchitan de Zaragoza, Oaxaca'
datos_csv.loc[datos_csv['ciudad'] == 'Juchitan de Zaragoza, Oaxaca', 'latitud'] = 16.440150611652577
datos_csv.loc[datos_csv['ciudad'] == 'Juchitan de Zaragoza, Oaxaca', 'longitud'] = -95.02226337475147

# Ajustar latitud y longitud para 'Ixtepec, Oaxaca'
datos_csv.loc[datos_csv['ciudad'] == 'Ixtepec, Oaxaca', 'latitud'] = 16.561107098248225
datos_csv.loc[datos_csv['ciudad'] == 'Ixtepec, Oaxaca', 'longitud'] = -95.0973463801423


### Agregamos columna 'etiqueta' con el Codigo de Ruta, para trabajar con nuestras rutas codificadas.

Tenemos que hacer que se codifique cada 'ciudad' dentro del df 'datos_csv', deacuerdo a su latitud y longitud y ala ruta ala que pertenece.

Cada 'ciudad' deacuerdo a su posicion en el mapa y a la columna 'ruta' a la que pertenece;

Ruta Sur = S
Ruta del Pacifico = P
Ruta Centro = C

In [37]:
# Paso 1: Calculamos la posición diagonal como la suma de latitud y longitud
datos_csv['posicion_diagonal'] = datos_csv['latitud'] + datos_csv['longitud']

# Paso 2: Mapeamos las rutas a sus códigos correspondientes
mapeo_rutas = {
    'Ruta Sur': 'S',
    'Ruta del Pacifico': 'P',
    'Ruta Centro': 'C',
    'Ruta del Golfo': 'G'
}
datos_csv['codigo_ruta'] = datos_csv['ruta'].map(mapeo_rutas)

# Paso 3: Ordenamos por ruta y posición diagonal, luego asignamos etiquetas numeradas dentro de cada grupo de ruta
datos_csv = datos_csv.sort_values(['codigo_ruta', 'posicion_diagonal'])

# Aquí generamos una nueva columna 'etiqueta' que combina el código de ruta con un número ascendente dentro de cada ruta
datos_csv['etiqueta'] = datos_csv.groupby('codigo_ruta').cumcount() + 1
datos_csv['etiqueta'] = datos_csv['codigo_ruta'] + datos_csv['etiqueta'].astype(str)

# Opcionalmente, podemos eliminar la columna 'Posicion_Diagonal' si ya no es necesaria
datos_csv.drop('posicion_diagonal', axis=1, inplace=True)

/var/folders/68/6zkv8z4j4rl6ds90br2rb7ph0000gn/T/ipykernel_771/3730677090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_csv['posicion_diagonal'] = datos_csv['latitud'] + datos_csv['longitud']
/var/folders/68/6zkv8z4j4rl6ds90br2rb7ph0000gn/T/ipykernel_771/3730677090.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_csv['codigo_ruta'] = datos_csv['ruta'].map(mapeo_rutas)


### Validamos Con Visualizacion

### Guardamos CSV general.

In [38]:
# Especificamos la ruta donde queremos guardar el archivo CSV
ruta_archivo = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Mapa_Casillas_2006_Reconstruido/Ciudades_Rutas/Ciudades_Rutas_Clase.csv'

# Guardamos el DataFrame en el archivo CSV en la ruta especificada
datos_csv.to_csv(ruta_archivo, index=False)

# Imprimimos un mensaje de confirmación
print("El DataFrame ha sido guardado exitosamente en:", ruta_archivo)


El DataFrame ha sido guardado exitosamente en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Mapa_Casillas_2006_Reconstruido/Ciudades_Rutas/Ciudades_Rutas_Clase.csv


### Y cada csv por ruta